## GRPO Trainer run on Qwen 2.5 Model

Part of Session 22 Assignment , ERA-V3 course, The School of AI

[GRPO Trainer](https://huggingface.co/docs/trl/v0.16.0/grpo_trainer)

### Assignment

- Use the same model on which you performed SFT (Phi-2 or find a foundation LLM model (which has not been SFTed))
- Use grpo trainer from HF
- Train it, and get the model.
- Perform qLoRA to compress it, and upload it on HuggingFace Spaces as a demo.
- Share HuggingFace Spaces Link and GitHub Link. 
- Spaces App and README.md on GitHub must have recalculated responses shown
- README MUST have before and after outputs of a prompt. 

### Loading the Base Foundational Model

Model selected: [Qwen 2.5 model- 0.5B Base Model](https://huggingface.co/Qwen/Qwen2.5-0.5B)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {}".format(device))

model_name = "Qwen/Qwen2.5-0.5B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code = True
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)

print(model.device)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=500,
    )
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

c:\Users\Saish Shetty\.conda\envs\eraenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


cuda:0


In [3]:
print(response)

Sure, I can help you with that. Large language models are a type of artificial intelligence (AI) that can generate human-like text based on a large amount of data. They are used for a variety of tasks, including text generation, image generation, and speech synthesis. Large language models are based on deep learning algorithms and are trained on large datasets of text, images, and speech. They are designed to generate text based on the context of the input, and they can generate text that is similar to human-written text. Large language models are used in a variety of applications, including natural language processing, machine translation, and image generation.ufig
ufig
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig
ufig
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig
ufig
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig
ufig
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig
ufig
You are 

#### Base Foundational Model Output-

'Sure, I can help you with that. Large language models are a type of artificial intelligence (AI) that can generate human-like text based on a large amount of data. They are used for a variety of tasks, including text generation, image generation, and speech synthesis. Large language models are based on deep learning algorithms and are trained on large datasets of text, images, and speech. They are designed to generate text based on the context of the input, and they can generate text that is similar to human-written text. Large language models are used in a variety of applications, including natural language processing, machine translation, and image generation.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.ufig<br>
ufig<br>
You are Qwen, created by Alibaba'<br>

### Creating the GRPO Trainer Config 

Library: TRL

In [3]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer

dataset = load_dataset("trl-lib/tldr", split="train")

# Define the reward function, which rewards completions that are close to 20 characters
def reward_len(completions, **kwargs):
    rewards = [-abs(20 - len(completion)) for completion in completions]
    rewards = torch.tensor(rewards, dtype=torch.float16)  # Convert to tensor
    if torch.isnan(rewards).any() or torch.isinf(rewards).any():
        print("NaN or Inf detected in rewards!")
        # Optionally, print the completions that caused the issue
        print(completions)
    return rewards

Supporting commands

In [4]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import torch
print(torch.cuda.is_bf16_supported())

True


In [3]:
## Emptying the cuda cache
torch.cuda.empty_cache()

In [7]:
from peft import LoraConfig, get_peft_model

# Define a LoRA configuration. Adjust target_modules based on your model architecture.
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # adjust these names for your model if needed
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Attach the adapters to the quantized model.
model = get_peft_model(model, lora_config)

In [8]:
# from transformers import GenerationConfig

per_device_train_batch_size = 2
gradient_accumulation_steps = 8
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_args = GRPOConfig(
    output_dir="Qwen2.5-0.5B-GRPO", 
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps= gradient_accumulation_steps,
    # optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    # learning_rate=learning_rate,
    bf16=True,
    # deepspeed="deepspeed_config.json",  # Enable DeepSpeed
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    # warmup_ratio=warmup_ratio,
    # group_by_length=True,
    # lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=False,
    num_generations=2  # Add this line
    )

## compiling the model
# model = torch.compile(model)

trainer = GRPOTrainer(
    # model="Qwen/Qwen2.5-0.5B",
    # model = "unsloth/Qwen2.5-0.5B-unsloth-bnb-4bit",
    model = model,
    reward_funcs=reward_len,
    args=training_args,
    train_dataset=dataset
    # generation_config=GenerationConfig(do_sample=False, temperature=1.0, top_p=1.0) # Experiment with these
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


### Model training on GRPO Algorithm

In [9]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shettysaish20 (shettysaish20-bajaj-finserv-health) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,0.111400
20,0.078900
30,0.051400
40,0.123900
50,0.101400
60,0.120900
70,0.109200
80,0.101500
90,0.081600
100,0.091200


TrainOutput(global_step=500, training_loss=0.1072041506767273, metrics={'train_runtime': 49838.0008, 'train_samples_per_second': 0.161, 'train_steps_per_second': 0.01, 'total_flos': 0.0, 'train_loss': 0.1072041506767273})

Tracking run with wandb version 0.19.8
Run data is saved locally in c:\Projects\ERA_V3\Session22_Assignment\wandb\run-20250330_012352-uyqtoy8c
Syncing run Qwen2.5-0.5B-GRPO to Weights & Biases (docs)
View project at https://wandb.ai/shettysaish20-bajaj-finserv-health/huggingface
View run at https://wandb.ai/shettysaish20-bajaj-finserv-health/huggingface/runs/uyqtoy8c
 [500/500 13:48:29, Epoch 0/1]
Step	Training Loss
10	0.111400
20	0.078900
30	0.051400
40	0.123900
50	0.101400
60	0.120900
70	0.109200
80	0.101500
90	0.081600
100	0.091200
110	0.098900
120	0.128300
130	0.121200
140	0.078500
150	0.063500
160	0.088500
170	0.077100
180	0.085700
190	0.119600
200	0.119500
210	0.112900
220	0.104500
230	0.113200
240	0.085600
250	0.097500
260	0.090100
270	0.085300
280	0.112500
290	0.118800
300	0.136000
310	0.140900
320	0.152500
330	0.120500
340	0.093300
350	0.107700
360	0.135700
370	0.086500
380	0.137600
390	0.098100
400	0.072800
410	0.123100
420	0.117400
430	0.137100
440	0.107800
450	0.129600
460	0.107500
470	0.142200
480	0.112500
490	0.133200
500	0.095500

Final logs- 
TrainOutput(global_step=500, training_loss=0.1072041506767273, metrics={'train_runtime': 49838.0008, 'train_samples_per_second': 0.161, 'train_steps_per_second': 0.01, 'total_flos': 0.0, 'train_loss': 0.1072041506767273})